# 🌠 [Day 4](https://adventofcode.com/2020/day/4)

In [1]:
import re

def passport_check_keys(lines):
    pattern = re.compile("(byr|iyr|eyr|hgt|hcl|ecl|pid|cid):(\S+)( |\n|$)")
    valid_keys = sorted(['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid'])
    num_valid_pwds = 0
    for line in lines:
        keys = [match.group(1) for match in re.finditer(pattern, line)
                if match.group(1) != 'cid']
        num_valid_pwds += int(sorted(keys) == valid_keys)
    return num_valid_pwds


def passport_check_format(lines):
    pattern = re.compile("(byr:[0-9]{4}"
                         "|iyr:[0-9]{4}"
                         "|eyr:[0-9]{4}"
                         "|hgt:[0-9]+(cm|in)"
                         "|hcl:#[a-f0-9]{6}"
                         "|ecl:(amb|blu|brn|gry|grn|hzl|oth)"
                         "|pid:[0-9]{9}"
                         "|cid:\S+)"
                         "( |\n|$)")
    valid_keys = ['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid']
    pwds = []
    for line in lines:
        p = {match[0][:3]: match[0][4:] 
             for match in re.findall(pattern, line)}
        # Check that all keys are present
        valid = True
        for k in valid_keys:
            if k not in p:
                valid = False
                break
        if not valid: continue
        # Check number ranges
        valid = ((1920 <= int(p['byr']) <= 2002) and
                 (2010 <= int(p['iyr']) <= 2020) and
                 (2020 <= int(p['eyr']) <= 2030) and
                 ((p['hgt'][-2:] == 'cm' and 150 <= int(p['hgt'][:-2]) <= 193) or
                  (p['hgt'][-2:] == 'in' and 59 <= int(p['hgt'][:-2]) <= 76))
                )
        if valid:
            pwds.append(p)
    return len(pwds), pwds

In [2]:
with open('inputs/day04.txt', 'r') as f:
    inputs = f.read().split('\n\n')
    
print(f"There are {passport_check_keys(inputs)} passports with valid keys")
print(f"But only {passport_check_format(inputs)[0]} valid passports")

There are 264 passports with valid keys
But only 224 valid passports


### Vizualisation with an attempt at using [pixelhouse](https://github.com/thoppe/pixelhouse)

In [3]:
from PIL import Image
import pixelhouse as ph
_, passports = passport_check_format(inputs)

In [4]:
def draw_passport(p, save=False):
    pal = ph.palette(1)
    # Basic scale unit, 1 = width / extent
    extent = 5
    # Main canvas
    canvas = ph.Canvas(400, 250, shift=0, extent=extent, bg=pal[-1])
    
    # Draw Photo frame
    offset = 1
    canvas += ph.rectangle(- 4.5, -2.5, -0.5, 2.5, color=pal[-3], thickness=-1)
    canvas += ph.rectangle(- 4.5, -2.5, -0.5, 2.5, color=pal[-2], thickness=0.2)
    
    # Determine y coordinate based on height
    min_y, max_y = 0.9, 1.2
    if p['hgt'][-2:] == 'cm':
        y = min_y + (max_y - min_y) * (int(p['hgt'][:-2]) - 150) / (193 - 150)
    else:
        y = min_y + (max_y - min_y) * (int(p['hgt'][:-2]) - 59) / (76 - 59)
        
    # Draw hair (back panel)
    canvas += ph.circle(-2.5, y + 0.3, r=0.85, color=p['hcl'])
    canvas += ph.rectangle(-3.35, y + 0.4, -1.65, y - 0.8, thickness=-1, color=p['hcl'])
    
    # Draw basic character head + body
    canvas += ph.circle(-2.5, y, r=0.8, color="black")
    r_body = 1
    canvas += ph.circle(-2.5, y - 0.9 - r_body, r=r_body, color="black")
    canvas += ph.rectangle(-2.5 - r_body, y - r_body - 0.9, 
                           -2.5  + r_body, -2.3, thickness=-1, color="black")
    
    # Draw hair bangs (front panel)
    xmin, xmax = -3.2, -1.8
    step = (xmax - xmin) / 3
    for x in range(3):
        canvas += ph.rectangle(xmin + x * step, y + 0.8,
                               xmin + (x + 1) * step - (0.1 if x < 2 else 0), y + 0.4,
                               thickness=-1, color=p['hcl'])
        
    # Draw eyes
    eye_color = ('darkgoldenrod' if p['ecl'] == 'amb' else
                 'mediumturquoise' if p['ecl'] == 'blu' else
                 'sienna' if p['ecl'] == 'brn' else
                 'lightgray' if p['ecl'] == 'gry' else
                 'springgreen' if p['ecl'] == 'grn' else
                 'sandybrown' if p['ecl'] == 'hzl' else
                 'mediumorchid' if p['ecl'] == 'oth' else None)
    for x in [-1, 1]:
        canvas += ph.circle(-2.5 + x * 0.3, y, r=0.12, color=eye_color)
        canvas += ph.circle(-2.5 + x * 0.3 - 0.01, y + 0.05, r=0.05, color='white')
        
    # Draw height bar / text
    canvas += ph.line(-2.5, y + 1.25, 1, y + 1.25, color='white', thickness=0.1)
    canvas += ph.text(p['hgt'], x=0.5, y= y + 1.55, font_size=0.35, color='white')
        
    # Draw text information on the right
    x = 2.5
    ystart = 1.5
    yshift = 0.6
    gap = 0.3
    canvas += ph.text(f"Country: {p.get('cid', '???')}", x=x, y=ystart,
                      font_size=0.45, color='white')
    canvas += ph.text(f"Age: {2020 - int(p['byr'])}", x=x, y=ystart - yshift,
                      font_size=0.45, color='white')
    canvas += ph.text(f"ID: {p['pid']}", x=x, y=ystart - 2 * yshift - gap,
                      font_size=0.45, color='white')
    canvas += ph.text(f"Issued: {p['iyr']}", x=x, y=ystart - 3 * yshift - gap,
                      font_size=0.45, color='white')
    canvas += ph.text(f"End: {p['eyr']}", x=x, y=ystart - 4 * yshift - gap,
                      font_size=0.45, color='white')
    # Save if needed
    if save:
        canvas.save("viz/day04.png")
    return Image.fromarray(canvas.img)
        
    
def draw_pasports_anim(ps):
    imgs = [draw_passport(p, save=False) for p in ps]
    imgs[0].save(fp="viz/day04.gif", format='GIF', append_images=imgs[1:],
                 save_all=True, duration=350, loop=0)
    
    
draw_passport(passports[1], save=True)
draw_pasports_anim(passports)

**Example passport generated**

![Passport example](viz/day04.png?2)

**Animation of all valid passports**

![Passport example](viz/day04.gif?2)